In [13]:
import re
import copy
import json
import graphviz
import numpy as np

In [14]:
def is_valid_regex(regex):
    """
    parameters :
    regex : check this regex if it valid or not
    return : 
    bool : True if it valid, and False if it invalid
    """
    try:
        re.compile(regex)
        return True
    except:
        return False

def add_concat(regex, operators):
    """
    add concatenate operator to given regex
    parameters :
    regex : valid regex
    operators : valid operators in regex 
    return : 
    regex_with_concat : valid regex with concatenate operator
    """
    regex_with_concat = []
    i = 0
    while i < len(regex) - 1:
        regex_with_concat.append(regex[i])
        if regex[i] not in operators:
            if regex[i + 1] not in operators or regex[i + 1] == '(' or regex[i + 1] == '[':
                regex_with_concat += '.'
        if (regex[i] == ')' or regex[i] == ']') and (regex[i + 1] == '(' or regex[i + 1] == '['):
            regex_with_concat += '.'
        if (regex[i] == '*' or regex[i] == '+' or regex[i] == '?') and (regex[i + 1] == '(' or regex[i + 1] == '['):
            regex_with_concat += '.'
        if (regex[i] == '*' or regex[i] == '+' or regex[i] == '?') and regex[i + 1] not in operators:
            regex_with_concat += '.'
        if (regex[i] == ')' or regex[i] == ']') and regex[i + 1] not in operators:
            regex_with_concat += '.'
        if regex[i] == '[':
            while regex[i+1] != ']':
                i += 1
                regex_with_concat.append(regex[i])
        i += 1

    regex_with_concat += regex[len(regex) - 1]
    return regex_with_concat

def comp_precedence(a, b):
    """
    parameters :
    a : first operator
    b : second operator
    return :
    bool : True if a has high precedence false otherwise
    """
    p = ["|", ".", "-", "*", "+", "?"]
    return p.index(a) > p.index(b)

def creat_postfix_regex(regex, operators):
    """
    apply Shunting-Yard algorithm to creat postfix regex
    parameters :
    regex : valid regex with concatenate operator
    operators : valid operators in regex 
    return : 
    postfix_regex : list of char that represente postfix regex 
    """
    postfix_regex = ""
    operator_stack = []

    for c in regex:
        if c not in operators or c == "*" or c == "+" or c == "?" or c == "[" or c == "]":
            postfix_regex += c
        elif c == ")": # or c == "]"
            while len(operator_stack) > 0 and operator_stack[-1] != "(" and operator_stack[-1] != "[":
                postfix_regex += operator_stack.pop()
            operator_stack.pop()
        elif c == "[" or c == "(" or len(operator_stack) == 0 or operator_stack[-1] == "(" or operator_stack[-1] == "[" or comp_precedence(c, operator_stack[-1]):
            operator_stack.append(c)
        else:
            while len(operator_stack) > 0 and operator_stack[-1] != "(" and operator_stack[-1] != "[" and not comp_precedence(c, operator_stack[-1]):
                postfix_regex += operator_stack.pop()
            operator_stack.append(c)

    while len(operator_stack) > 0:
        postfix_regex += operator_stack.pop()

    return postfix_regex



In [15]:
class state:
    def __init__(self, id="", is_start=False, is_terminal=False, next_states={}):
        self.id = id
        self.is_start = is_start
        self.is_terminal = is_terminal
        self.next_states = next_states
    def print_state(self):
        print("state ", self.id)
        for out in self.next_states:
            print("on ", out, "go to", self.next_states[out])
        if self.is_start:
            print("state is a start state")
        if self.is_terminal:
            print("state is a terminal state")
    
    def __str__(self):
        return self

In [16]:
def creat_nfa(postfix_regex, operators):
    """
    convert postfix regex to finite state machine using Thompson's Construction algorithm for NFAs
    parameters :
    postfix_regex : postfix regex (output from Shunting-Yard algorithm)
    operators : valid operators in regex 
    return : 
    states : array holds all states that formulate NFA 
    """
    i = 0
    states = []
    states_stack =[]
    while i < len(postfix_regex):
        if postfix_regex[i] not in operators:
            s1 = state(str(i) + str(0), True, False, {postfix_regex[i] : str(i) + str(1)})
            s2 = state(str(i) + str(1), False, True, {})
        else:
            if postfix_regex[i] == '[':
                next_states = {}
                state_id = i
                if len(postfix_regex) > i+4 and postfix_regex[i+4] == '-':
                    for j in range(ord(postfix_regex[i+1]), ord(postfix_regex[i+2]) + 1):
                        next_states[chr(j)] = str(state_id) + str(1)
                    i += 4 
                else:
                    while postfix_regex[i+1] != ']':
                        i += 1
                        next_states[postfix_regex[i]] = str(state_id) + str(1)
                    i += 1
                s1 = state(str(state_id) + str(0), True, False, next_states)
                s2 = state(str(state_id) + str(1), False, True, {})

            if postfix_regex[i] == '|':
                second_nfa_state = states_stack.pop()
                first_nfa_state = states_stack.pop()
                s1 = state(str(i) + str(0), True, False, {"epsilon" : [first_nfa_state[0].id, second_nfa_state[0].id]})
                s2 = state(str(i) + str(1), False, True, {})
                # add epsilon
                if "epsilon" in first_nfa_state[1].next_states:
                    first_nfa_state[1].next_states["epsilon"].append(s2.id)
                else:
                    first_nfa_state[1].next_states["epsilon"] = [s2.id]
                if "epsilon" in second_nfa_state[1].next_states:
                    second_nfa_state[1].next_states["epsilon"].append(s2.id)
                else:
                    second_nfa_state[1].next_states["epsilon"] = [s2.id]
                # make it non trminal
                first_nfa_state[1].is_terminal = False
                second_nfa_state[1].is_terminal = False

                # make it non start state
                first_nfa_state[0].is_start = False
                second_nfa_state[0].is_start = False

            if postfix_regex[i] == '.':
                second_nfa_state = states_stack.pop()
                first_nfa_state = states_stack.pop()
                s1 = state(str(i) + str(0), True, False, {"epsilon" : [first_nfa_state[0].id]})
                s2 = state(str(i) + str(1), False, True, {})
                
                # add epsilon
                if "epsilon" in first_nfa_state[1].next_states:
                    first_nfa_state[1].next_states["epsilon"].append(second_nfa_state[0].id)
                else:
                    first_nfa_state[1].next_states["epsilon"] = [second_nfa_state[0].id]
                if "epsilon" in second_nfa_state[1].next_states:
                    second_nfa_state[1].next_states["epsilon"].append(s2.id)
                else:
                    second_nfa_state[1].next_states["epsilon"] = [s2.id]
                
                # make it non trminal
                first_nfa_state[1].is_terminal = False
                second_nfa_state[1].is_terminal = False
                
                # make it non start state
                first_nfa_state[0].is_start = False
                second_nfa_state[0].is_start = False

            if postfix_regex[i] == '*':
                nfa_state = states_stack.pop()
                s1 = state(str(i) + str(0), True, False, {"epsilon" : [nfa_state[0].id, str(i) + str(1)]})
                s2 = state(str(i) + str(1), False, True, {})
                
                # add epsilon
                if "epsilon" in nfa_state[1].next_states:
                    nfa_state[1].next_states["epsilon"].append(nfa_state[0].id)
                else:
                    nfa_state[1].next_states["epsilon"] = [nfa_state[0].id]
                nfa_state[1].next_states["epsilon"].append(s2.id)
                
                # make it non trminal
                nfa_state[1].is_terminal = False

                # make it non start state
                nfa_state[0].is_start = False

            if postfix_regex[i] == '+':
                nfa_state = states_stack.pop()
                s1 = state(str(i) + str(0), True, False, {"epsilon" : [nfa_state[0].id]})
                s2 = state(str(i) + str(1), False, True, {})
                
                # add epsilon
                if "epsilon" in nfa_state[1].next_states:
                    nfa_state[1].next_states["epsilon"].append(nfa_state[0].id)
                else:
                    nfa_state[1].next_states["epsilon"] = [nfa_state[0].id]
                nfa_state[1].next_states["epsilon"].append(s2.id)
                
                # make it non trminal
                nfa_state[1].is_terminal = False

                # make it non start state
                nfa_state[0].is_start = False
            
            if postfix_regex[i] == '?':
                nfa_state = states_stack.pop()
                s1 = state(str(i) + str(0), True, False, {"epsilon" : [nfa_state[0].id, str(i) + str(1)]})
                s2 = state(str(i) + str(1), False, True, {})
                
                # add epsilon
                if "epsilon" in nfa_state[1].next_states:
                    nfa_state[1].next_states["epsilon"].append(s2.id)
                else:
                    nfa_state[1].next_states["epsilon"] = [s2.id]
                
                # make it non trminal
                nfa_state[1].is_terminal = False

                # make it non start state
                nfa_state[0].is_start = False
        
        states_stack.append((s1, s2))
        states.append(s1)
        states.append(s2)
        i += 1
    return states

def write_nfa_states_json(states):
    """
    write all NFA states in json file and draw a graph for all states
    parameters :
    states : array holds all states that formulate NFA 
    return :
    dict_states : dictionary key is states id and value is states
    starting_state : starting state in nfa
    """
    dot = graphviz.Digraph()
    data = {
    "startingState": "bla",
    }

    starting_state = None 
    dict_states = {}
    for state in states:
        dict_states[state.id] = state
        if state.is_start and state.is_terminal:
            dot.node(state.id, shape= "doublecircle")
            data["startingState"] = state.id
            starting_state = state.id
        elif state.is_start:
            data["startingState"] = state.id
            starting_state = state.id
            dot.node(state.id)
        elif state.is_terminal:
            dot.node(state.id, shape= "doublecircle")
        else:
            dot.node(state.id)
        
        if state.is_start:
            data["startingState"] = state.id
            starting_state = state.id
        state_data = {
            "isTerminatingState": state.is_terminal
        }
        for next_state in state.next_states:
            state_data["inputCharacter" + next_state.upper()] = state.next_states[next_state]
            if isinstance(state.next_states[next_state], list):
                for edge in state.next_states[next_state]:
                    dot.edge(state.id, edge, next_state)
            else:
                dot.edge(state.id, state.next_states[next_state], next_state)
        data["state" + state.id] = state_data

    with open("NFA.json", "w") as f:
        json.dump(data, f)
    
    dot.render('NFA')
    return dict_states, starting_state

def re_to_nfa(regex):
    """
    convert regex to NFA
    parameters :
    regex : input regex 
    return :
    dict_states : dictionary key is states id and value is states
    starting_state : starting state in nfa
    """
    if not is_valid_regex(regex):
        return "error, invalid regex"
    operators = ['|', '+', '*', '?', '-', '(', ')', '[', ']', '.']
    regex_with_concat = add_concat(regex, operators)
    postfix_regex = creat_postfix_regex(regex_with_concat, operators)
    states = creat_nfa(postfix_regex, operators)
    dict_states, starting_state = write_nfa_states_json(states)
    return dict_states, starting_state
    

In [31]:
def load_nfa_states_json(filename = "NFA"):
    """
    load all NFA states from json file 
    parameters :
    filename : json file name to be loaded
    return :
    dict_states : dictionary key is states id and value is states
    starting_state : starting state in nfa
    """
    file = open(filename + '.json')
    data = json.load(file)
    
    dict_states = {}
    for i in data:
        if i == "startingState":
            starting_state = data["startingState"]
        else:
            id = i.replace("state",'')
            is_terminal = data[i]["isTerminatingState"]
            next_states = {}
            for j in data[i]:
                if "inputCharacter" in j:
                    action = j.replace("inputCharacter", '')
                    action = action.lower()
                    next_states[action] = data[i][j]
            s = state(id, id == starting_state, is_terminal, next_states)
            dict_states[id] = s
        
    file.close()
    return dict_states, starting_state

def epsilon_closure(state, nfa_states):
    """
    loop over epsilon paths from given state to get epsilon closure
    parameters :
    state : given state id to begin with this state
    nfa_states : dictionary of all nfa states; key : state id, value : state
    return :
    state_with_epsilon : a new state that holds all states id that presents the epsilon closure
    is_terminal : result from oring is_terminal to each state in new state
    """
    visited_states = []
    unvisited_states = [state]
    state_with_epsilon = []
    is_terminal = False
    # epsilon closure on start state
    while len(unvisited_states) > 0:
        current_state = nfa_states[unvisited_states[0]]
        visited_states.append(unvisited_states[0])
        unvisited_states = unvisited_states[1:]
        is_terminal = is_terminal or current_state.is_terminal
        state_with_epsilon.append(current_state.id)
        for next_state in current_state.next_states:
            if next_state == "epsilon":
                for dest_state in current_state.next_states[next_state]:
                    if dest_state not in visited_states:
                        unvisited_states.append(dest_state)


    return state_with_epsilon, is_terminal

def nfa_to_dfa(nfa_states, starting_state):
    """
    parameters :
    nfa_states : dictionary of all nfa states; key : state id, value : state
    starting_state : start state id in NFA state machine
    return :
    dfa_states_dict : dictionary that represent all DFA states; key : id, value : state
    set_of_actions : set of all different actions from state to other
    """
    set_of_actions = set()
    for nfa_state in nfa_states:
        for next_state in nfa_states[nfa_state].next_states:
            set_of_actions.add(next_state)
    set_of_actions.remove('epsilon')

    state_index = 0
    # epsilon closure on start state
    state_with_epsilon, is_terminal = epsilon_closure(starting_state, nfa_states)
    starting_state_with_epsilon = set(state_with_epsilon)
    s = state(str(state_index), True, is_terminal, {})

    dfa_states_ids = [starting_state_with_epsilon]
    dfa_states_dict = {state_index : s}
    queue = [starting_state_with_epsilon]
    while len(queue) > 0:
        current_dfa_state = queue.pop(0)
        for action in set_of_actions:
            new_state = set()
            new_is_terminal = False
            for nfa_state in current_dfa_state:
                if action in nfa_states[nfa_state].next_states:
                    new_state.add(nfa_states[nfa_state].next_states[action])
                    state_with_epsilon, is_terminal = epsilon_closure(nfa_states[nfa_state].next_states[action], nfa_states)
                    new_is_terminal = new_is_terminal or is_terminal
                    new_state = set(state_with_epsilon).union(new_state)
                    # new_state = set(new_state)

            if len(new_state) > 0 and new_state not in dfa_states_ids:
                dfa_states_ids.append(new_state)
                queue.append(new_state)
                s = state(str(dfa_states_ids.index(new_state)), False, new_is_terminal, {})
                dfa_states_dict[dfa_states_ids.index(new_state)] = s
            
            if len(new_state) > 0:
                dfa_states_dict[state_index].next_states[action] = str(dfa_states_ids.index(new_state))
        state_index += 1

    return dfa_states_dict, set_of_actions

def write_dfa_states_json(states, is_minimized):
    """
    write all DFA states in json file
    parameters :
    states : array holds all states that formulate DFA 
    is_minimized : flag if the dfa is minimized : True
    """
    dot = graphviz.Digraph()
    data = {
    "startingState": "bla",
    }

    for state in states:
        if states[state].is_start and states[state].is_terminal:
            dot.node(states[state].id, "start", shape= "doublecircle")
            data["startingState"] = states[state].id
        elif states[state].is_start:
            data["startingState"] = states[state].id
            dot.node(states[state].id, "start")
        elif states[state].is_terminal:
            dot.node(states[state].id, shape= "doublecircle")
        else:
            dot.node(states[state].id)

        state_data = {
            "isTerminatingState": states[state].is_terminal
        }
        for next_state in states[state].next_states:
            state_data["inputCharacter" + next_state.upper()] = states[state].next_states[next_state]
            if isinstance(states[state].next_states[next_state], list):
                for edge in state.next_states[next_state]:
                    dot.edge(states[state].id, edge, next_state)
            else:
                dot.edge(states[state].id, states[state].next_states[next_state], next_state)
        data["state" + states[state].id] = state_data
    
    if is_minimized:
        file_name = "min_DFA.json"
        dot.render('min_DFA')
    else:
        file_name = "DFA.json"
        dot.render('DFA')

    with open(file_name, "w") as f:
        json.dump(data, f)


In [26]:
def minimize_dfa(dfa_states, set_of_actions):
    """
    apply minimized algorithm to reduce number of DFA states
    parameters :
    dfa_states : dictionary of all DFA states; key : state id, value : state
    set_of_actions : set of all different actions from state to other
    return :
    minimized_states_dict : dictionary of all minimized DFA states; key : state id, value : state
    """
    terminal_state = []
    nonterminal_state = []
    for dfa_state in dfa_states:
        if dfa_states[dfa_state].is_terminal:
            terminal_state.append(dfa_states[dfa_state].id)
        else:
            nonterminal_state.append(dfa_states[dfa_state].id)
    
    if len(terminal_state) == 0:
        minimized_states = [nonterminal_state]
    elif len(nonterminal_state) == 0:
        minimized_states = [terminal_state]
    else:
        minimized_states = [terminal_state, nonterminal_state]
    is_updated = True
    while is_updated:
        is_updated = False
        new_minimized_states = copy.deepcopy(minimized_states)
        for i in range(len(minimized_states)):
            if len(minimized_states[i]) != 1:
                for action in set_of_actions:
                    destination_state = None
                    # this loop determine which state should be destination state
                    for minimized_state in minimized_states[i]:
                        if action in dfa_states[int(minimized_state)].next_states:
                            destination_state = dfa_states[int(minimized_state)].next_states[action]
                            break
                    if destination_state:
                        for i in range(len(minimized_states)):
                            if destination_state in minimized_states[i]:
                                destination_state_index = i
                                break
                        out_states = []
                        for dfa_state in minimized_states[i]:
                            if action not in dfa_states[int(dfa_state)].next_states or dfa_states[int(dfa_state)].next_states[action] not in minimized_states[destination_state_index]:
                                out_states.append(dfa_state)
                                new_minimized_states[i].remove(dfa_state)
                                is_updated = True
                    if is_updated:
                        new_minimized_states.append(out_states)
                        break
        minimized_states = new_minimized_states

    minimized_states_dict = {}
    ids = []
    for minimized_state in minimized_states:
        ids.append(''.join(set(minimized_state)))

    for minimized_state in minimized_states:
        id = ""
        is_terminal = False
        is_start = False
        next_states = {}
        for one_state in set(minimized_state):
            id += dfa_states[int(one_state)].id
            is_terminal = is_terminal or dfa_states[int(one_state)].is_terminal
            is_start = is_start or dfa_states[int(one_state)].is_start
            for next_state in dfa_states[int(one_state)].next_states:
                if dfa_states[int(one_state)].next_states[next_state] in ids:
                    next_states[next_state] = dfa_states[int(one_state)].next_states[next_state]
                else:
                    for one_id in ids:
                        if dfa_states[int(one_state)].next_states[next_state] in one_id:
                            next_states[next_state] = one_id
                            break
        minimized_states_dict[id] = state(id, is_start, is_terminal, next_states)
    return minimized_states_dict


### try the algorithm :) 
##### enter any regex to convert it to minimized DFA

In [40]:
nfa_dict_states, starting_state = re_to_nfa("ab(b|c)*d+") 
# nfa_dict_states, starting_state = load_nfa_states_json("NFA")
dfa_states, set_of_actions = nfa_to_dfa(nfa_dict_states, starting_state)
write_dfa_states_json(dfa_states, False)

minimized_states_dict = minimize_dfa(dfa_states, set_of_actions)
write_dfa_states_json(minimized_states_dict, True)